# DUI DATA
Deep Urban Interaction - Data Preparation  
Ryan Yan Zhang <ryanz@mit.edu>  
City Science, MIT Media Lab  

In [1]:
# General Imports
from pprint import pprint
import random
import datetime
import time

from IPython.core.debugger import set_trace

import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Data Generation

## Image from video with OpenCV

## Pose json from image with OpenPose

## People Bounding box from OpenPose

# Data Editing

## Resize and Stitch 3 Images

In [ ]:
import cv2
import numpy as np
import json
from pprint import pprint
import os

# constant parameters
xx = 0
yy = 1

# input parameters
img_path = '../data/raw/DUI/valid_original_png'
json_path = '../data/raw'
img_out_path = r'C:\Users\RYAN\Documents\GitHub\CS_Urban_Interaction_CV\interaction-analysis-python\02_interaction-classification\PyTorch_RZ\data\viz\04_bounding_box_images'
json_label_path = r'C:\Users\RYAN\Documents\GitHub\CS_Urban_Interaction_CV\interaction-analysis-python\02_interaction-classification\PyTorch_RZ\data\viz\05_bounding_box_jsons'
fps = 30
joint_confident_threshold = 0.1
d_expand_fixed = 50.0

# main code
if not os.path.exists(img_path):
    sys.exit('img_path doesn\'t exist!')
if not os.path.exists(json_path):
    sys.exit('json_path doesn\'t exist!')
if not os.path.exists(img_out_path):
    os.makedirs(img_out_path)
    print(f'made path: {img_out_path}')
if not os.path.exists(json_label_path):
    os.makedirs(json_label_path)
    print(f'made path: {json_label_path}')

    
# get file list. Make sure that the image names' order is corespondent to the json file names' order!!!
img_file_names = [f for f in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, f))]
json_file_names = [f for f in os.listdir(json_path) if os.path.isfile(os.path.join(json_path, f))]


# loop all the image files, json files, and save result
for input_image_file_name in os.listdir(img_path):
    if (input_image_file_name.endswith(".png") or input_image_file_name.endswith(".jpg")):
        #print(os.path.join(img_path, input_image_file_name))
        input_json_file_name = os.path.splitext(input_image_file_name)[0] + "_keypoints.json"
        #print(os.path.join(json_path, input_json_file_name))
        output_image_file_name = os.path.splitext(input_image_file_name)[0] + "_tagged.png"
        #print(os.path.join(img_out_path, output_image_file_name))
        json_label_file_name = input_image_file_name + ".json"
        
        # Load an color image
        img = cv2.imread(os.path.join(img_path, input_image_file_name))
        # read the json file
        with open(os.path.join(json_path, input_json_file_name)) as f:
            json_data = json.load(f)
        
        # get data of effective joints of people in one json file
        joint_confident_threshold = 0.1
        # loop person in one json
        people_list = []
        for i in range(len(json_data["people"])):
            # loop joints of one person
            effective_joint_xy_list = []
            for j in range(int(len(json_data["people"][i]["pose_keypoints"])/3)):
                if json_data["people"][i]["pose_keypoints"][j*3+2] > joint_confident_threshold:
                    effective_joint_xy_list.append([json_data["people"][i]["pose_keypoints"][j*3+xx], json_data["people"][i]["pose_keypoints"][j*3+yy]])
            people_list.append(effective_joint_xy_list)
        
        # analysis to extrapolate the bounding box
        # simple max min bounding box of all the joints plus expand distance joint 0 to joint 1 for all 4 side
        # loop person
        box_list = []
        for i in range(len(people_list)):
            box = [[9999999,9999999],[0,0]] #[upper_left_xy,lower_right_xy]
            joint_list = people_list[i]
            d_expand = d_expand_fixed

            # loop joint xy
            for j in range(len(joint_list)):
                joint_xy = joint_list[j]
                box = [[min(box[0][xx],joint_xy[xx]),min(box[0][yy],joint_xy[yy])],[max(box[1][xx],joint_xy[xx]),max(box[1][yy],joint_xy[yy])]]
            # expend the box by distance joint 0 to joint 1 for all 4 side
            box = [[box[0][xx]-d_expand,box[0][yy]-d_expand],[box[1][xx]+d_expand,box[1][yy]+d_expand]]
            box_list.append(box)
        
        # draw the boxes
        for i in range(len(box_list)):
            box = box_list[i]
            cv2.rectangle(img,(int(box[0][xx]),int(box[0][yy])),(int(box[1][xx]),int(box[1][yy])),(255,255,255),2)
        
        # write the image in to file
        cv2.imwrite(os.path.join(img_out_path, output_image_file_name), img)
        print('{} is saved'.format(output_image_file_name))
        
        # construct json_label
        data = {}
        data['filename'] = input_image_file_name
        data['file_ext'] = os.path.splitext(input_image_file_name)[1]
        data['file_path'] = img_path
        height, width, channels = img.shape
        data['file_dim'] = [width, height]
        data['people'] = []
        for i in range(len(box_list)):
            box = box_list[i]
            pplData = {}
            pplData['tl_coord'] = [int(box[0][xx]),int(box[1][yy])]
            pplData['br_coord'] = [int(box[1][xx]),int(box[0][yy])]
            pplData['dims'] = [int(box[1][xx]-box[0][xx]), int(box[1][yy]-box[0][yy])]
            pplData['area'] = int(box[1][xx]-box[0][xx]) * int(box[1][yy]-box[0][yy])
            data['people'].append(pplData)
        data['count'] = len(box_list)
        
        with open(os.path.join(json_label_path, json_label_file_name), 'w') as outfile:
            json.dump(data, outfile, indent=4)
        
        #t = t + 1  # run only first t-1 files
        continue
    else:
        #t = t + 1  # run only first t-1 files
        continue